In [2]:
from tensorflow import keras
import tensorflow as tf

In [3]:
import tensorflow_datasets as tfds

In [4]:
dataset,info = tfds.load("tf_flowers",as_supervised=True,with_info=True)
dataset_size = info.splits["train"].num_examples#3670
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes#5

In [4]:
train_data = tfds.load('tf_flowers', split='train[:10%]', as_supervised=True)
valid_data = tfds.load('tf_flowers', split='train[10%:25%]', as_supervised=True)
test_data = tfds.load('tf_flowers', split='train[25%:]', as_supervised=True)

In [5]:
def preprocess(image,label):
  resized_image = tf.image.resize(image,[224,224])
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image,label

In [6]:
batch_size = 32
train_set = train_data.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_data.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_data.map(preprocess).batch(batch_size).prefetch(1)

In [7]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense(n_classes,activation="softmax")(avg)
loc_output = keras.layers.Dense(4)(avg)
model = keras.Model(inputs=base_model.input,
                    outputs=[class_output,loc_output])
optimizer = tf.keras.optimizers.legacy.SGD(lr=0.2,momentum=0.9,decay=0.01)
model.compile(loss=["sparse_categorical_crossentropy","mse"],
              loss_weights=[0.8,0.2],
              optimizer=optimizer,
              metrics=["accuracy"])

83683744/83683744 [==============================] - 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
history = model.fit(train_set,epochs=5,validation_data=valid_set)

Epoch 1/5
12/12 [==============================] - 30s 1s/step - loss: 1.6376 - dense_loss: 1.2622 - dense_1_loss: 3.1392 - dense_accuracy: 0.5531 - dense_1_accuracy: 0.2207 - val_loss: 1453613.0000 - val_dense_loss: 13549.2715 - val_dense_1_loss: 7213866.5000 - val_dense_accuracy: 0.1815 - val_dense_1_accuracy: 0.2686
Epoch 2/5
12/12 [==============================] - 8s 660ms/step - loss: 1.5359 - dense_loss: 1.3671 - dense_1_loss: 2.2111 - dense_accuracy: 0.5395 - dense_1_accuracy: 0.2044 - val_loss: 26862428160.0000 - val_dense_loss: 637156.9375 - val_dense_1_loss: 134309576704.0000 - val_dense_accuracy: 0.1815 - val_dense_1_accuracy: 0.1742
Epoch 3/5
12/12 [==============================] - 8s 662ms/step - loss: 1.5093 - dense_loss: 1.4061 - dense_1_loss: 1.9222 - dense_accuracy: 0.3924 - dense_1_accuracy: 0.1771 - val_loss: 7819571494912.0000 - val_dense_loss: 12490775.0000 - val_dense_1_loss: 39097808715776.0000 - val_dense_accuracy: 0.1815 - val_dense_1_accuracy: 0.2377
Epoch 4

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

# Load the data
data = tfds.load("tf_flowers", split="train[:90%]", as_supervised=True)
test_data = tfds.load("tf_flowers", split="train[90%:]", as_supervised=True)
train_data = data.take(2090)
valid_data = data.skip(2090)

# Normalize the images and resize them to (224, 224)
def preprocess(image, label):
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.xception.preprocess_input(image)
  return image, label

train_data = train_data.map(preprocess).batch(32)
valid_data = valid_data.map(preprocess).batch(32)
test_data = test_data.map(preprocess).batch(32)

# Load the Xception model and add new layers
base_model = Xception(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.inputs, outputs=predictions)

# Freeze the convolutional layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=5, validation_data=valid_data)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data)
print('Test accuracy:', test_acc)


Epoch 1/5
66/66 [==============================] - 27s 290ms/step - loss: 0.5450 - accuracy: 0.8077 - val_loss: 0.3688 - val_accuracy: 0.8722
Epoch 2/5
66/66 [==============================] - 13s 195ms/step - loss: 0.2365 - accuracy: 0.9167 - val_loss: 0.3642 - val_accuracy: 0.8755
Epoch 3/5
66/66 [==============================] - 13s 204ms/step - loss: 0.1505 - accuracy: 0.9512 - val_loss: 0.4486 - val_accuracy: 0.8524
Epoch 4/5
66/66 [==============================] - 14s 207ms/step - loss: 0.1148 - accuracy: 0.9636 - val_loss: 0.7171 - val_accuracy: 0.8030
Epoch 5/5
12/12 [==============================] - 3s 210ms/step - loss: 0.3332 - accuracy: 0.8856
Test accuracy: 0.8855586051940918
